# II. Identifying Received Transactions

This section begins to disect a Monero transation. Alice recieved a transaction from an unknown person, and later sent a transaction to Bob. The ultimate goal is to understand every single portion of a transaction. In this section every portion of a transaction relevant to the receiver identifying ownership will be decoded and explained.

This section relies heavliy on the following source:
* [Zero to Monero: Second Edition; Chapter 4](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf)


**It is highly recommended that the reader read chapter 4 of _Zero to Monero: Second Edition_ prior to reading the following sections.**

The sections below cover:
1. extracting relevant data from the transaction
    1. stealth addresses (K)
    2. transaction public key (rG)
    3. encrypted payment id
2. calculating view tags
3. identify owned outputs
4. decrypting payment id
5. decrypting payment amount

### Summary (Introduction)

**What we need (previousy calculated):**
* Alice's private view key: `e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`

**What we start with (from the transaction data):**
* view tag
* stealth address (K)
* transaction public key (rG)
* encrypted payment id
* encrypted payment amount

**What we end up with:**
* decrypted payment id
* decrypted payment amount

## 1. Extracting relevant data from the transaction

Each transaction contains three pieces of information that are used to determine if the transaction was addressed to them. The three pieces of data are:

* view tag: one byte of data that allows a quick check to see if the transaction is *likely* to be Alice's
    * included in the `vout` section of the transaction
* stealth address: one-time public addresses
    * included in the `vout` section of the transaction
* transaction public key: usually written as `rG`
    * included in the `extra` section of the transaction

Transactions also contain payment ids (similar to a memo section on a check):

* encrypted payment id
    * included in the `extra` section of the transaction

And transactions contain payment amounts:

* encrypted amount
    * included in the `rct_signatures` \ `ecdhInfo` section of the transaction

In [1]:
# read data directly from the saved transaction
import json

with open("../../transactions/inbound/txn_be71.json", "r") as file:
    txn = json.load(file)

vout = txn["vout"]
txextra = txn["extra"]
amount = txn["rct_signatures"]["ecdhInfo"]

The `extra` section can contain arbitrary data so rules are used to convert specific data in the field into known purposes. Refer back to Zero to Monero for further clarity on the rules. In this transaction the `extra` field contains the transaction public key and the encrypted payment id. Both will be extracted.

In [2]:
'''
Zero to Monero: Second Edition; Appendix A
'''


'''
To extract data from extra:

  1. Extracts 32 bytes of data for the transaction public key and converts to hex
  2. Extracts 8 more bytes of data for the payment id and converts to hex
'''

# read initial 32 bytes of data from the extra field
rG = bytes(txextra[1:33]).hex()

# read an additional 8 bytes of data from the extra field
encrypted_payment_id = bytes(txextra[36:44]).hex()


print(f"Transaction public key (rG): {rG}")
print(f"Encrypted payment id: {encrypted_payment_id}")
print(f"Field (extra): {txextra}")

Transaction public key (rG): 587a7d33f0b28ee85cfeeee0da7cab9ff5c348a648fee350099f95ccf2e3cc39
Encrypted payment id: 4a0094809c81cdb9
Field (extra): [1, 88, 122, 125, 51, 240, 178, 142, 232, 92, 254, 238, 224, 218, 124, 171, 159, 245, 195, 72, 166, 72, 254, 227, 80, 9, 159, 149, 204, 242, 227, 204, 57, 2, 9, 1, 74, 0, 148, 128, 156, 129, 205, 185]


The `vout` section contains a list of items, with one item per recipient. Every transaction has at least two items in the list. In the most common case one of the two items is for the recipient and the other item is change being returned to the sender. As will be shown later the first list item was sent to Alice. The second list item will be used to show that Alice can see it is not addressed to her.

In [3]:
'''
Zero to Monero: Second Edition; Appendix A
'''

# read view tags from each of the two transaction outputs 
view_tag = vout[0]["target"]["tagged_key"]["view_tag"]
view_tag_not_alice = vout[1]["target"]["tagged_key"]["view_tag"]

# read stealth addresses from each of the two transaction outputs
stealth_address = vout[0]["target"]["tagged_key"]["key"]
stealth_address_not_alice = vout[1]["target"]["tagged_key"]["key"]


print(f"First output view tag: {view_tag}")
print(f"First output stealth address: {stealth_address}")
print(f"Field (vout): {json.dumps(vout, indent=4)}")

First output view tag: 2a
First output stealth address: 6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374
Field (vout): [
    {
        "amount": 0,
        "target": {
            "tagged_key": {
                "key": "6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374",
                "view_tag": "2a"
            }
        }
    },
    {
        "amount": 0,
        "target": {
            "tagged_key": {
                "key": "9cb1ec2a60e305dbb65c575c9bef4157ca1db7ae8d0a581aceb6f5c3855ad299",
                "view_tag": "04"
            }
        }
    }
]


The `rct_signatures` section contains a section named `ecdhInfo` with a list of items, with one item per recipient. Every transaction has at least two items in the list. In the most common case one of the two items is for the recipient and the other item is change being returned to the sender. As will be shown later the first list item was sent to Alice. The second list item will be used to show that Alice can see it is not addressed to her.

In [4]:
'''
Zero to Monero: Second Edition; Appendix A
'''

# read view tags from each of the two transaction outputs 
amount_1 = amount[0]["amount"]
amount_2 = amount[1]["amount"]

print(f"First output encrypted amount: {amount_1}")
print(f"Field (rct_signatures\ecdhInfo): {json.dumps(amount, indent=4)}")

First output encrypted amount: 51769df0938cf016
Field (rct_signatures\ecdhInfo): [
    {
        "amount": "51769df0938cf016"
    },
    {
        "amount": "8dec59af8eff53f1"
    }
]


## 2. Calculating view tags

A recently added feature to Monero is a single byte value included in a transaction that allows Alice to quickly "check" to see if an output is *likely* addressed to her. This quick check skips more computationally intensive calculations later. Including view tags drastically reduces the amount of time required to scan through millions of transactions on the blockchain.

In [5]:
'''
Not present in  Zero to Monero: Second Edition
'''


'''
To calculate the view tag this code performs the following:

  1. Multiplies the private view key by the transaction public key
  2. Concatenates the string "view_tag" with the value created in step 1 and
     an index identifying the output number (uses zero indexing)
  3. Hashes this value with Keccack
  4. Extracts the first byte from this hash
'''

import varint
import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


# define a function to return a hashed byte string
def keccak_256(data) -> bytes:
    return keccak.new(digest_bits=256).update(data).digest()

# define a function that allows multiplying a curve point by a scalar (eg: integer)
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp


# previously calculated from Alice's mnemonic
private_view_key = "e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b"

# Monero multiplies curve points by 8, create 8 in bytes for use in calculations
cofactor = int(8).to_bytes(32, byteorder="little")

# multiply the private view key by rG to obtain rKv
rKv = scalar_point_mult(unhexlify(private_view_key), unhexlify(rG))

# multiply by 8, required by Monero
rKv = scalar_point_mult(cofactor, rKv)

# concatenate the string "view_tag", rKv, and a transaction index integer
rKv_concat_extra_index = b"view_tag" + rKv + varint.encode(0)

# hash the previously concatenated value to create H(rKv)
HrKv = keccak_256(rKv_concat_extra_index)

# the view tag is the first byte of this hashed string
view_tag_calculated = HrKv[0:1]


print(f"First output view tag (calculated): {hexlify(view_tag_calculated).decode()}")
print(f"First output view tag (from txn data): {view_tag}")

First output view tag (calculated): 2a
First output view tag (from txn data): 2a


For clarity the view tag for the second output will be calculated. The calculated view tag should **not** match transaction data because the second output is not addressed to Alice. The only difference in the calculation between the two outputs is that the transaction index changed from 0 to 1 for the second output.

In [6]:
# concatenate the string "view_tag", rKv, and a transaction index integer
rKv_concat_extra_index = b"view_tag" + rKv + varint.encode(1)

# hash the previously concatenated value to create H(rKv)
HrKv = keccak_256(rKv_concat_extra_index)

# the view tag is the first byte of this hashed string
view_tag_calculated = HrKv[0:1]


print(f"Second output view tag (calculated): {hexlify(view_tag_calculated).decode()}")
print(f"Second output view tag (from txn data): {view_tag_not_alice}")

Second output view tag (calculated): 68
Second output view tag (from txn data): 04


## 3. Identifying owned outputs

Alice has identified that the first output is *likely* to be hers. To confirm this is true she will perform calculations to explicitly confirm the output was addressed to her. To do this she will perform calculations with the stealth address and then compare the result to her public spend key. If the result matches her public spend key then the output meant for her. If the output was not destined for her, this calculation would **not** reveal the public spend key for anyone else (the calculation results would be useless).

In [7]:
'''
Zero to Monero: Second Edition; Section 4.2
'''


'''
To identify owned outputs this code performs the following:

  1. Multiplies the private view key by the transaction public key
  2. Concatenates the value created in step 1 and an index identifying 
     the output number (uses zero indexing)
  3. Hashes this value with Keccack
  4. Multiplies this by G to create a new curve point
  5. Subtracts this point from the stealth address to obtain an expected
     public spend key
'''

import nacl.bindings


# define a function to return a hashed byte string
def keccak_256(data):
    return keccak.new(digest_bits=256).update(data).digest()

# define a function to convert an integer into a valid ed25519 scalar
def pad_and_reduce(scalar: bytes) -> bytes:
    point_padded_to_64_bytes = scalar + (64 - len(scalar)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)

# define a function to multiply the ed25519 generator point G by a scalar (eg: integer)
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp

# define a function to add to curve points
point_add = nacl.bindings.crypto_core_ed25519_add

# define a function to subtract curve point p2 from curve point p1
point_sub = nacl.bindings.crypto_core_ed25519_sub


# previously calculated from Alice's mnemonic
public_spend_key = "cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37"


### begin section: this is repeated from the view tag section for clarity
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
private_view_key = "e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b"
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(private_view_key), unhexlify(rG))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(0)
HrKv = keccak_256(rKv_concat_extra_index)
### end section


# confirm hash is a valid curve scalar
HrKv = pad_and_reduce(HrKv)

# multiply the scalar by the ed25519 generator point to obtain a new curve point
HrKvG = scalar_mult_G(HrKv)

# calculate an expected public spend key: K's = stealth address - H(rKv)G
public_spend_key_calculated = point_sub(unhexlify(stealth_address), HrKvG)

# recalculate the stealth address for clarity: stealth address  = H(rKv)G + public_spend_key
stealth_address_calculated = point_add(HrKvG, unhexlify(public_spend_key))


print(f"Public spend key (calculated): {hexlify(public_spend_key_calculated).decode()}")
print(f"Public spend key (actual): {public_spend_key}")

Public spend key (calculated): cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37
Public spend key (actual): cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37


Intuition behind stealth addresses:

Imagine a person named Prince wants to send Monero to Alice but wants to creat a new address that only that person and Alice know. Prince has access to Alice's normal address and thus her public spend and view keys. Alice of course has access to her public/private spend/view keys.

Prince wants to build up a shared secret so he starts with what he has (public keys) and works towards that goal with the following reasoning:
1. Prince could hash Alice's public view key, ensure the hash is a valid curve scalar and then multiply that by G to get a new public key. That would be a new address but anyone with Alice's public view key would be able to derive everything. That's not a shared secret.
2. Prince creates a random number `r` and calculates `rK`. Alice can calculate `k(rG)`. Both of those are the same, but Alice only needs `rG`. It is not possible to recover `r` from `rG` so Prince can put `rG` in transaction data without worrying that someone else can calcualate `r`. 
    * Why is it important to hide `r` from outside observers? 
        * Anyone with Alice's Monero address has access to her public keys, and using `r` could recreate the stealth address: H(rKv) + Ks. That's not a shared secret. But hiding `r` creates a shared secret.
    * For clarity:
        * `rK` = `rkG` = `krG`
        * Adding parenthesis for clarity: `rK` = `r(kG)` = `k(rG)`
3. Prince adds Alice's public spend key to the prior data to create `H(rKv)G` + `Ks`. Both Prince and Alice can create this address. This value is equal to [`H(rKv)` + `ks`]`G` which means the private key is `H(rKv)` + `ks`. Prince has created an address that only he and Alice can create but with a private key that only Alice can create.
    * What's wrong with just using `H(rKv)G`?
        * The private key for `K(rKv)G` is the portion before the final `G`, `H(rKv)`. Prince obviously has access to that which means Prince would be able to spend that output. That's not good.


An evil third party who has access to Alice's address has her public keys (`Kv`, `Ks`), and `rG` from the transaction data. What can they calculate? `rKvG` + `Ks`. **That is completely different from `rkvG` + `Ks` (the value Alice calculated). In the same way it is completely different from `rKv` + `Ks` (the value Prince calculated).

There are two pieces of data used and the reason for those are as follows:
* `r`: The value r is hidden from evil third parties but can be used directly by Prince and indirectly (rG) by Alice to create the same result.
* `Ks`: The public spend key is added because Prince does not know the corresponding private key and thus cannot calculate the stealth address private key.

Prince: Easily creates the one time address and publishes the stealth address [`H(rKv)G` + `Ks`], and `rG`.
Alice: Uses `rG` to recreate [`H(rkvG)G` + `Ks`], and the corresponding private key.
Evil third party: Cannot use `rG` to recreate the value even if they have access to Alice's public keys.

For clarity the same calculations will be performed on the second stealth address. The calculated public spend key should **not** match Alice's public spend key because the second output is not addressed to Alice. Similar to the second view tag calculation, the only difference in the calculation between the two outputs is that the transaction index changed from 0 to 1 for the second output.

As a reminder, the expected public spend key created here is **not** the true recipient's public spend key.

In [8]:
# concatenate rKv and an index corresponding to the transaction number (ex: first transaction = 0, second transaction = 1, etc)
rKv_concat_extra_index = rKv + varint.encode(1)

# hash the results: H(rKv) = (8 * r * public_view_key) + transaction_index = rKv || transaction_index
HrKv = keccak_256(rKv_concat_extra_index)

# confirm hash is a valid curve scalar
HrKv = pad_and_reduce(HrKv)

# multiply the scalar by the ed25519 generator point to obtain a new curve point
HrKvG = scalar_mult_G(HrKv)

# K's = K0 - H(rKv)G
public_spend_key_calculated = point_sub(unhexlify(stealth_address_not_alice), HrKvG)
print(f"Public spend key (calculated): {hexlify(public_spend_key_calculated).decode()}")
print(f"Public spend key (actual): {public_spend_key}")

Public spend key (calculated): 9b4607f964ddd978604f3eb268221ce39fa699e732d2eebfbddd3a0fbe338b77
Public spend key (actual): cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37


## 4. Decrypting payment id

Monero transactions always include a payment id. If a payment id is not specified by the sender the payment id should be all zeros but encrypted in such a way that only the sender and Alice can recover it. 

In [9]:
'''
Zero to Monero: Second Edition; Section 4.4 (but applied to regular addresses not integrated addresses)
'''


'''
To recover the paymend id this code performs the following:

  1. Concatenates the value rKv and a Monero constant (the integer 141 as bytes)
  2. Creates an obfuscation mask from the value in step 1
  3. Recovers the payment id by applying bitwise xor operation with the value from step 2 
'''

# define a function to calculate exclusive or between bits in two variables
def calculate_xor(var1, var2):
    return bytearray(a ^ b for a, b in zip(*map(bytearray, (var1, var2))))


# convert the number 141 to bytes 
encrypted_payment_id_tail = int(141).to_bytes()

# create obfuscation mask
payment_id_mask = rKv + encrypted_payment_id_tail
payment_id_mask = keccak_256(payment_id_mask)

# convert encrypted payment id to bytes
encrypted_payment_id_bytes = bytes.fromhex(encrypted_payment_id)

# truncate obfuscation mask to length of encrypted payment id
truncated_payment_id_mask = payment_id_mask[: len(encrypted_payment_id_bytes)]

# apply binary xor operation to encrypted payment id
decrypted_payment_id_bytes = calculate_xor(truncated_payment_id_mask, encrypted_payment_id_bytes)


print(f"Encrypted payment id: {encrypted_payment_id}")
print(f"Decrypted payment id: {decrypted_payment_id_bytes.hex()}")

Encrypted payment id: 4a0094809c81cdb9
Decrypted payment id: 0000000000000000


## 5. Decrypting payment amount

The amount Alice received is encoded in the `amount` field. It is encoded by using the shared secret `HrKv` to encrypt/decrypt the data. The amount is also contained in another field that will be addressed in a later step. 

In [10]:
'''
Zero to Monero: Second Edition; Section 5.3 (not including the commitment part)
'''


'''
To recover the payment amount this code performs the following:

  1. Recreates HrKv
  2. Concatentates the string "amount" to the value from step 1
  3. Hashes the value in step 2 to create a mask
  4. Truncates the mask to the length of the amount field
  5. Applies bitwise xor to the amount field using the mask
  6. Converts the results to an integer to represent piconeros
'''

from decimal import Decimal


# define a function to convert an integer into a valid ed25519 scalar
def pad_and_reduce(point):
    point_padded_to_64_bytes = point + (64 - len(point)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)


piconero = Decimal("0.000000000001")


### begin section: this is repeated from the view tag section for clarity
scalar_point_mult = nacl.bindings.crypto_scalarmult_ed25519_noclamp
private_view_key = "e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b"
cofactor = int(8).to_bytes(32, byteorder="little")
rKv = scalar_point_mult(unhexlify(private_view_key), unhexlify(rG))
rKv = scalar_point_mult(cofactor, rKv)
rKv_concat_extra_index = rKv + varint.encode(0)
HrKv = keccak_256(rKv_concat_extra_index)
### end section

# ensure the hash is a valid scalar
amount_mask = pad_and_reduce(HrKv)

# calculate Keccak hash of concatenated "amount" and scalar
amount_mask = keccak_256(b"amount" + amount_mask)

# truncate obfuscation mask to length of amount field
truncated_amount_mask = amount_mask[: len(unhexlify(amount_1))]

# apply binary xor operation to encrypted amount
decrypted_amount = calculate_xor(unhexlify(amount_1), truncated_amount_mask)

# convert decrypted amount to an integer and convert units to Monero instead of piconero
decrypted_amount = int.from_bytes(decrypted_amount, byteorder="little")

print(f"Encrypted amount: {amount_1}")
print(f"Decrypted amount (piconero): {decrypted_amount}")
print(f"Decrypted amount (Monero): {decrypted_amount * piconero}")

Encrypted amount: 51769df0938cf016
Decrypted amount (piconero): 69300000
Decrypted amount (Monero): 0.000069300000



## Summary (Conclusion)
**What we need (previousy calculated):**
* Alice's private view key: `e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b`
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37`
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867`

**What we start with (from the transaction data):**
* view tag: `2a`
* stealth address: `6f598be2c3c473ccff7ad1fb42ed46660bae0ea353d71192afe1a520aacb9374`
* rG: `587a7d33f0b28ee85cfeeee0da7cab9ff5c348a648fee350099f95ccf2e3cc39`
* encrypted payment id: `4a0094809c81cdb9`
* encrypted amount: `51769df0938cf016`

**What we end up with:**
* decrypted payment id: `0000000000000000`
* decrypted amount (piconero): `69300000`